In [1]:
using MultivariatePolynomials
const MP = MultivariatePolynomials
# The names ares stored outside so that `isbits(::SpinVariable)` is `true`.
const NAMES = String[]
struct SpinVariable <: MP.AbstractVariable
    id::Int # Spin id, spins with different id commute
    index::Int # 0 means x, 1 means y and 2 means z
end
function spin(name::String)
    push!(NAMES, name)
    id = length(NAMES)
    return SpinVariable(id, 0), SpinVariable(id, 1), SpinVariable(id, 2)
end

spin (generic function with 1 method)

In [2]:
function MP.name_base_indices(var::SpinVariable) # Used to print variable
    splits = split(NAMES[var.id], r"[\[,\]]\s*", keepempty=false)
    suffix = ("ˣ", "ʸ", "ᶻ")[var.index + 1]
    name = splits[1] * suffix
    if length(splits) == 1
        return name, Int[]
    else
        return name, parse.(Int, splits[2:end])
    end
end

In [3]:
function Base.:*(a::SpinVariable, b::SpinVariable)
    if a.id == b.id
        if a.index == b.index
            return true # We want to return `1` but in which type ?
                        # We use `Bool` type as it the type compatible with the most other types in Julia.
        else
            i = a.index
            j = b.index
            if j == (i + 1) % 3
                # σx * σy = im * σz
                return (true * im) * SpinVariable(a.id, (j + 1) % 3)
            else
                # σy * σx = 0
                return false # Same comment than for `return true` above.
            end
        end
    else
        error("TODO")
    end
end

In [4]:
function spin_index(prefix::String, indices)
    return spin(prefix * "[" * join(indices, ",") * "]")
end
function array_spin(prefix, indices...)
    σs = map(i -> spin_index(prefix, i), Iterators.product(indices...))
    return [σ[1] for σ in σs], [σ[2] for σ in σs], [σ[3] for σ in σs]
end

function build_spin(var)
    if isa(var, Symbol)
        σx = Symbol(string(var) * "x")
        σy = Symbol(string(var) * "y")
        σz = Symbol(string(var) * "z")
        return [σx, σy, σz], :(($(esc(σx)), $(esc(σy)), $(esc(σz))) = spin($"$var"))
    else
        isa(var, Expr) || error("Expected $var to be a variable name")
        Base.Meta.isexpr(var, :ref) || error("Expected $var to be of the form varname[idxset]")
        (2 ≤ length(var.args)) || error("Expected $var to have at least one index set")
        varname = var.args[1]
        prefix = string(varname)
        σx = Symbol(prefix * "x")
        σy = Symbol(prefix * "y")
        σz = Symbol(prefix * "z")
        return [σx, σy, σz], :(($(esc(σx)), $(esc(σy)), $(esc(σz))) = array_spin($prefix, $(esc.(var.args[2:end])...)))
    end
end

function build_spins(args)
    vars = Symbol[]
    exprs = []
    for arg in args
        var, expr = build_spin(arg)
        append!(vars, var)
        push!(exprs, expr)
    end
    return vars, exprs
end

# Variable vector x returned garanteed to be sorted so that if p is built with x then vars(p) == x
macro spin(args...)
    vars, exprs = build_spins(args)
    :($(foldl((x,y) -> :($x; $y), exprs, init=:())); $(Expr(:tuple, esc.(vars)...)))
end

@spin (macro with 1 method)

In [5]:
spin("σ")

(σˣ, σʸ, σᶻ)

In [6]:
spin_index("σ", 1)

(σˣ₁, σʸ₁, σᶻ₁)

In [7]:
@macroexpand @spin σ

quote
    begin
        ()
        #= In[4]:42 =#
        (σx, σy, σz) = Main.spin("σ")
    end
    #= In[4]:42 =#
    (σx, σy, σz)
end

In [8]:
@spin σ

(σˣ, σʸ, σᶻ)

In [9]:
σx * σx

true

In [10]:
σy * σy

true

In [11]:
σy * σx

false

In [12]:
σx * σy # TODO

MethodError: MethodError: no method matching monomial(::SpinVariable)
Closest candidates are:
  monomial(!Matched::AbstractMonomial) at /home/blegat/.julia/packages/MultivariatePolynomials/goWHi/src/term.jl:129

In [13]:
@spin σ[1:2]

(SpinVariable[σˣ₁, σˣ₂], SpinVariable[σʸ₁, σʸ₂], SpinVariable[σᶻ₁, σᶻ₂])

In [14]:
σx[1] * σx[2] # TODO

ErrorException: TODO